In [1]:
import modulos as md

lista=['XRPUSDT',
'EOSUSDT',
'LTCUSDT',
'ADAUSDT',
'BNBUSDT',
'DOGEUSDT',
'KAVAUSDT',
'RLCUSDT',
'SOLUSDT',
'STORJUSDT',
'TOMOUSDT',
'MATICUSDT',
'ALPHAUSDT',
'SANDUSDT',
'LINAUSDT',
'MTLUSDT',
'MASKUSDT',
'DYDXUSDT',
'ARPAUSDT',
'OPUSDT',
'INJUSDT',
'LDOUSDT',
'APTUSDT',
'RNDRUSDT',
'CFXUSDT',
'STXUSDT',
'ARBUSDT',
'SUIUSDT',
'RADUSDT'
]

for symbol in lista:
    data = md.obtiene_historial(symbol)
    data = md.estrategia(data)
    md.backtesting(data,symbol)

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


EOSUSDT - 5.187310245764024
ADAUSDT - 1.1295585718008851
STORJUSDT - 4.514127433442127
MATICUSDT - 0.17157481010326592
ALPHAUSDT - 3.4612016261847884
LINAUSDT - 0.5264804594127099
CFXUSDT - 2.1522178780178365
STXUSDT - 34.84600199356214
SUIUSDT - 41.03080129422756
